In [1]:
from keras.models import Sequential, save_model, load_model
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
# Hyper parameters
batch_size = 128
nb_epoch = 10

# Parameters for MNIST dataset
nb_classes = 10

# Parameters for MLP
prob_drop_input = 0.2               # drop probability for dropout @ input layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer

In [3]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_Train = np_utils.to_categorical(y_train, nb_classes)
Y_Test = np_utils.to_categorical(y_test, nb_classes)

In [4]:
# Multilayer Perceptron model
model = Sequential()
model.add(Dense(output_dim=625, input_dim=784, init='normal', activation='sigmoid', name='dense1'))
model.add(Dropout(prob_drop_input, name='dropout1'))
model.add(Dense(output_dim=625, input_dim=625, init='normal', activation='sigmoid', name='dense2'))
model.add(Dropout(prob_drop_hidden, name='dropout2'))
model.add(Dense(output_dim=10, input_dim=625, init='normal', activation='softmax', name='dense3'))
model.compile(optimizer=RMSprop(lr=0.001, rho=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense1 (Dense)                   (None, 625)           490625      dense_input_1[0][0]              
____________________________________________________________________________________________________
dropout1 (Dropout)               (None, 625)           0           dense1[0][0]                     
____________________________________________________________________________________________________
dense2 (Dense)                   (None, 625)           391250      dropout1[0][0]                   
____________________________________________________________________________________________________
dropout2 (Dropout)               (None, 625)           0           dense2[0][0]                     
___________________________________________________________________________________________

In [5]:
# Train
save_model(model, './logs/model_mlp')
checkpoint = ModelCheckpoint(filepath='./logs/weights.epoch.{epoch:02d}-val_loss.{val_loss:.2f}.hdf5', verbose=0)
history = model.fit(X_train, Y_Train, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1,
                    callbacks=[checkpoint], validation_data=(X_test, Y_Test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s - loss: 0.5620 - acc: 0.8227 - val_loss: 0.2750 - val_acc: 0.9141
Epoch 2/10
60000/60000 [==============================] - 3s - loss: 0.2497 - acc: 0.9242 - val_loss: 0.1697 - val_acc: 0.9485
Epoch 3/10
60000/60000 [==============================] - 3s - loss: 0.1791 - acc: 0.9457 - val_loss: 0.1320 - val_acc: 0.9585
Epoch 4/10
60000/60000 [==============================] - 3s - loss: 0.1430 - acc: 0.9561 - val_loss: 0.1216 - val_acc: 0.9623
Epoch 5/10
60000/60000 [==============================] - 3s - loss: 0.1181 - acc: 0.9636 - val_loss: 0.0966 - val_acc: 0.9700
Epoch 6/10
60000/60000 [==============================] - 3s - loss: 0.1015 - acc: 0.9694 - val_loss: 0.0895 - val_acc: 0.9730
Epoch 7/10
60000/60000 [==============================] - 3s - loss: 0.0888 - acc: 0.9721 - val_loss: 0.0846 - val_acc: 0.9746
Epoch 8/10
60000/60000 [==============================] - 3s 

In [6]:
# Evaluate
evaluation = model.evaluate(X_test, Y_Test, verbose=1)
print('\nSummary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

 9920/10000 [============================>.] - ETA: 0s
Summary: Loss over the test dataset: 0.08, Accuracy: 0.98


In [7]:
# Restore trained model
loaded_model = load_model('./logs/model_mlp')
loaded_model.load_weights('./logs/weights.epoch.09-val_loss.0.08.hdf5')
loaded_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense1 (Dense)                   (None, 625)           490625      dense_input_2[0][0]              
____________________________________________________________________________________________________
dropout1 (Dropout)               (None, 625)           0           dense1[0][0]                     
____________________________________________________________________________________________________
dense2 (Dense)                   (None, 625)           391250      dropout1[0][0]                   
____________________________________________________________________________________________________
dropout2 (Dropout)               (None, 625)           0           dense2[0][0]                     
___________________________________________________________________________________________

In [8]:
# Evaluate with loaded model
evaluation = loaded_model.evaluate(X_test, Y_Test, verbose=1)
print('\nSummary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

 9952/10000 [============================>.] - ETA: 0s
Summary: Loss over the test dataset: 0.08, Accuracy: 0.98
